In [1]:
import pybullet as p
import pybullet_data
import cv2
import random
import cv2
import numpy as np
import os
import pandas as pd

In [2]:
#define joint ranges
min, max= -np.pi/2, np.pi/2
joint_ranges = [(min, max), (min, max), (min, max),(min, max),(min, max),(min, max)]

In [3]:
#Top Camera Setup
width, height = 640, 480
top_camera = p.computeViewMatrixFromYawPitchRoll(cameraTargetPosition=[0, 0, 0],distance=2.5,yaw=0, pitch=-90, roll=0, upAxisIndex=2)
top_projection = p.computeProjectionMatrixFOV(fov=60, aspect=float(width) / height, nearVal=0.1, farVal=100.0)

# Side Camera Setup
side_camera_position = [2, 0, .5]  
side_camera_target = [0, 0, .5]      # Point the camera is looking at
side_camera_up_vector = [0, 0, 1]   # Which direction is 'up' for the camera
side_camera = p.computeViewMatrix(cameraEyePosition=side_camera_position, cameraTargetPosition=side_camera_target, cameraUpVector=side_camera_up_vector)
side_projection = p.computeProjectionMatrixFOV(fov=60, aspect=float(width) / height, nearVal=0.1, farVal=100.0) # Define the side camera projection matrix

#Front Camera Setup
front_camera_position = [0, 2, .5] 
front_camera_target = [0, 0, .5]      # Point the camera is looking at
front_camera_up_vector = [0, 0, 1]   # Which direction is 'up' for the camera
front_camera = p.computeViewMatrix(cameraEyePosition=front_camera_position, cameraTargetPosition=front_camera_target, cameraUpVector=front_camera_up_vector)
front_projection = p.computeProjectionMatrixFOV(fov=60, aspect=float(width) / height, nearVal=0.1, farVal=100.0) # Define the side camera projection matrix

#corner 1 Camera Setup
corner1_camera_position = [1.5, 1.5, 1]  
corner1_camera_target = [0, 0, .5]      # Point the camera is looking at
corner1_camera_up_vector = [0, 0, 1]   # Which direction is 'up' for the camera
corner1_camera = p.computeViewMatrix(cameraEyePosition=corner1_camera_position, cameraTargetPosition=corner1_camera_target, cameraUpVector=corner1_camera_up_vector)
corner1_projection = p.computeProjectionMatrixFOV(fov=60, aspect=float(width) / height, nearVal=0.1, farVal=100.0)

#corner 2 Camera Setup
corner2_camera_position = [1.5, -1.5, -1]  
corner2_camera_target = [0, 0, .5]      # Point the camera is looking at
corner2_camera_up_vector = [0, 0, 1]   # Which direction is 'up' for the camera
corner2_camera = p.computeViewMatrix(cameraEyePosition=corner2_camera_position, cameraTargetPosition=corner2_camera_target, cameraUpVector=corner2_camera_up_vector)
corner2_projection = p.computeProjectionMatrixFOV(fov=60, aspect=float(width) / height, nearVal=0.1, farVal=100.0)

In [14]:
#connect to a pybulllet sim
if p.isConnected():
    p.disconnect()
p.connect(p.GUI)
p.setGravity(0, 0, -9.8)
p.setAdditionalSearchPath(pybullet_data.getDataPath())
plane_id= p.loadURDF("plane.urdf") #load a plane
arm_id = p.loadURDF("assets/ur5.urdf", [0, 0, 0.4],p.getQuaternionFromEuler([0, 0, 0]))
obstacle1= p.loadURDF('assets/block.urdf',basePosition=[0, 0.65, 0.9],useFixedBase=True)
obstacle2= p.loadURDF('assets/block.urdf',basePosition=[.6, 0.95, .3],useFixedBase=True)
obstacle3= p.loadURDF('assets/block.urdf',basePosition=[-.6, -0.75, 0.9],useFixedBase=True)
obstacle4= p.loadURDF('assets/block.urdf',basePosition=[-.7,-0.65, .3],useFixedBase=True)

#create a directory for images
images_dir = "UR5_images_GodMode"
os.makedirs(images_dir, exist_ok=True)

def get_depth_image(depth_buffer, near, far, width, height):
    depth = far * near / (far - (far - near) * depth_buffer)
    depth_image = np.array(depth * 255, dtype=np.uint8).reshape(height, width)
    return depth_image

# Initialize DataFrame
columns = ['ImageID', 'Joint1', 'Joint2', 'Joint3', 'Joint4', 'Joint5', 'Joint6']
positions_df = pd.DataFrame(columns=columns)

temp_data = []

for i in range(10000):

    joint_positions= []

    # Set random pose
    for joint_index, (min_val, max_val) in enumerate(joint_ranges):
        random_position = random.uniform(min_val, max_val)
        p.setJointMotorControl2(bodyUniqueId=arm_id,
                                jointIndex=joint_index,
                                controlMode=p.POSITION_CONTROL,
                                targetPosition=random_position)
        joint_positions.append(random_position)
        
    # Allow some time for the arm to move to the pose
    for _ in range(100):
        p.stepSimulation()

    # Capture and save images
    # _, _, top_image, _, _ = p.getCameraImage(width, height, top_camera, top_projection)
    # _, _, side_image, _, _ = p.getCameraImage(width, height, side_camera, side_projection)
    # _, _, front_image, _, _ = p.getCameraImage(width, height, front_camera, front_projection)
    # _, _, corner1_image, _, _ = p.getCameraImage(width, height, corner1_camera, corner1_projection)
    # _, _, corner2_image, _, _ = p.getCameraImage(width, height, corner2_camera, corner2_projection)

    top_image_path = os.path.join(images_dir, f"top_view_{i}.png")
    side_image_path = os.path.join(images_dir, f"side_view_{i}.png")
    front_image_path = os.path.join(images_dir, f"front_view_{i}.png")
    corner1_image_path = os.path.join(images_dir, f"corner1_view_{i}.png")
    corner2_image_path = os.path.join(images_dir, f"corner2_view_{i}.png")

    # cv2.imwrite(top_image_path, top_image)
    # cv2.imwrite(side_image_path, side_image)
    # cv2.imwrite(front_image_path, front_image)
    # cv2.imwrite(corner1_image_path, corner1_image)
    # cv2.imwrite(corner2_image_path, corner2_image)
    
    for camera_config, image_path_prefix in zip(
        [(top_camera, top_projection), (side_camera, side_projection), 
         (front_camera, front_projection), (corner1_camera, corner1_projection), 
         (corner2_camera, corner2_projection)],
        ['top', 'side', 'front', 'corner1', 'corner2']):
        # Capture RGB and Depth images
        _, _, rgba_image, depth_buffer, _ = p.getCameraImage(width, height, camera_config[0], camera_config[1])
        
        # Convert RGBA to RGB (discard alpha channel)
        rgb_image = rgba_image[:, :, :3]
        
        # Convert depth buffer to depth image
        depth_image = get_depth_image(np.array(depth_buffer), 0.1, 100.0, width, height)

        # Save RGB and Depth images
        cv2.imwrite(os.path.join(images_dir, f"{image_path_prefix}_view_{i}.png"), rgb_image)
        cv2.imwrite(os.path.join(images_dir, f"{image_path_prefix}_depth_view_{i}.png"), depth_image)



    temp_data.append({'ImageID': i, 'Joint1': joint_positions[0], 'Joint2': joint_positions[1], 'Joint3': joint_positions[2], 
                      'Joint4': joint_positions[3], 'Joint5': joint_positions[4], 'Joint6': joint_positions[5]})

# Concatenate the temporary data to the DataFrame
positions_df = pd.concat([positions_df, pd.DataFrame(temp_data)], ignore_index=True)
  
# Append data to DataFrame
positions_df.to_csv('UR5_positions_GodMode.csv', index=False)
